In [2]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

model=Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

input_tensor=Input(shape=(64,))
x=layers.Dense(32,activation='relu')(input_tensor)
x=layers.Dense(32,activation='relu')(x)
output_tensor=layers.Dense(10,activation='softmax')(x)

model=Model(input_tensor,output_tensor)

model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy')

import numpy as np
X_train=np.random.random((1000,64))
y_train=np.random.random((1000,10))

model.fit(X_train,y_train,epochs=10,batch_size=128)
score=model.evaluate(X_train,y_train)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1000/1000 [==============================] - 0s 149us/step - loss: 11.4920
Epoch 2/10
1000/1000 [==============================] - 0s 16us/step - loss: 11.4535
Epoch 3/10
1000/1000 [==============================] - 0s 24us/step - loss: 11.4438
Epoch 4/10
1000/

In [6]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

text_vocabulary_size=10000
question_vocabulary_size=10000
answer_vocabulary_size=500

text_input=Input(shape=(None,),dtype='int32',name='text')

embedded_text=layers.Embedding(text_vocabulary_size,64)(text_input)

encoded_text=layers.LSTM(32)(embedded_text)

question_input=Input(shape=(None,),dtype='int32',name='question')
embedded_question=layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question=layers.LSTM(16)(embedded_question)

concatenated=layers.concatenate([encoded_text,encoded_question],axis=-1)

answer=layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

model=Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

In [8]:
import numpy as np
num_samples=1000
max_length=100

text=np.random.randint(1,text_vocabulary_size,size=(num_samples,max_length))
question=np.random.randint(1,question_vocabulary_size,size=(num_samples,max_length))

answers=np.zeros(shape=(num_samples,answer_vocabulary_size))
indices=np.random.randint(0,answer_vocabulary_size,size=num_samples)
for i,x in enumerate(answers):
    x[indices[i]]=1
    
model.fit([text,question],answers,epochs=10,batch_size=128)

model.fit({'text':text,'question':question},answers,epochs=10,batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2143 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1975 - acc: 0.0350
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1419 - acc: 0.0090
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0586 - acc: 0.0060
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0049 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9577 - acc: 0.0070
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9000 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8270 - acc: 0.0100
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7469 - acc: 0.0180
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6497 - acc: 0.0290
Epoch 1/10
1000/1000 [=======

In [16]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

vocabulary_size=50000
num_income_groups=10

posts_input=Input(shape=(None,),dtype='int32',name='posts')
embedded_posts=layers.Embedding(256,vocabulary_size)(posts_input)
x=layers.Conv1D(128,5,activation='relu')(embedded_posts)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.GlobalMaxPooling1D()(x)
x=layers.Dense(128,activation='relu')(x)

age_prediction=layers.Dense(1,name='age')(x)
income_prediction=layers.Dense(num_income_groups,activation='softmax',name='income')(x)
gender_prediction=layers.Dense(1,activation='sigmoid',name='gender')(x)

model=Model(posts_input,[age_prediction,income_prediction,gender_prediction])

model.compile(optimizer='rmsprop',
              loss=['mse',
                   'binary_crossentropy',
                   'categorical_crossentropy'],
              loss_weights=[0.25,1.,10.])

model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income':'categorical_crossentropy',
                    'gender':'binary_crossentropy'},
              loss_weights={'age':0.25,'income':1.,'gender':10.})

model.summary()

model.fit(posts,[age_targets,income_targets,gender_targets])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, None, 128)    32000128    embedding_11[0][0]               
__________________________________________________________________________________________________
max_pooling1d_13 (MaxPooling1D) (None, None, 128)    0           conv1d_31[0][0]                  
__________________________________________________________________________________________________
conv1d_32 

NameError: name 'posts' is not defined